<a href="https://colab.research.google.com/github/jish1398/Nlu_Intent/blob/master/nlu_intents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!apt-get -qq install -y graphviz libgraphviz-dev pkg-config && pip install pygraphviz;
!pip install rasa_nlu[tensorflow];
!python -m spacy download en;
!pip install git+https://github.com/mit-nlp/MITIE.git
!pip install rasa_nlu[mitie]


    Linking successful
    /usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
    /usr/local/lib/python3.6/dist-packages/spacy/data/en

    You can now load the model via spacy.load('en')

  Cloning https://github.com/mit-nlp/MITIE.git to /tmp/pip-req-build-mx4s5vwo
  Running setup.py bdist_wheel for mitie ... - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ done
  Stored in directory: /tmp/pip-ephem-wheel-cache-mlvoqi0l/wheels/b4/c1/21/8e7e7e14cf3211bf5c73aad0b1d76d1186fbf681f4b9ef6c06
Successfully built mitie


Create the database and try different 

In [0]:
nlu_md = """

## intent:button
- please click(Action) the [red](color) button
- click(Action) button 
- click(Action) the blue button 
- click(Action) the [black](color) button 
- click(Action) the [first](position) button
- click(Action) the [third](position) [red](color) button
- click(Action) the [second](position) button
- press(Action) the blue button
- Please go to(Action) the next page by clicking(Action) the [red](color) button

## lookup:Action   <!-- lookup table list -->
- press
- go to
- click
- clicking

## intent:mood_unhappy
- my day was horrible
- I am sad
- I don't feel very well
- I am disappointed
- super sad
- I'm so sad
- sad
- very sad
- unhappy
- bad
- very bad
- awful
- terrible
- not so good
- not very good
- extremly sad
- so saad
- so sad

"""
%store nlu_md > nlu.md

Writing 'nlu_md' (str) to file 'nlu.md'.


In [0]:
nlu_config = """
language: en
pipeline: 
- name: "tokenizer_whitespace"
- name: "ner_crf"
- name: "ner_synonyms"
- name: "intent_featurizer_count_vectors"
- name: "intent_classifier_tensorflow_embedding"
  intent_tokenization_flag: true
  intent_split_symbol: "+"
  """
%store nlu_config > nlu_config.yml


Writing 'nlu_config' (str) to file 'nlu_config.yml'.


In [0]:
!python -m rasa_nlu.train -c nlu_config.yml --data nlu.md -o models --fixed_model_name nlu --project current --verbose

2018-10-31 03:24:24 INFO     rasa_nlu.training_data.loading  - Training data format of nlu.md is md
2018-10-31 03:24:24 INFO     rasa_nlu.training_data.training_data  - Training data stats: 
	- intent examples: 27 (2 distinct intents)
	- Found intents: 'mood_unhappy', 'button'
	- entity examples: 6 (2 distinct entities)
	- found entities: 'color', 'position'

2018-10-31 03:24:24 INFO     rasa_nlu.model  - Starting to train component tokenizer_whitespace
2018-10-31 03:24:24 INFO     rasa_nlu.model  - Finished training component.
2018-10-31 03:24:24 INFO     rasa_nlu.model  - Starting to train component ner_crf
2018-10-31 03:24:24 INFO     rasa_nlu.model  - Finished training component.
2018-10-31 03:24:24 INFO     rasa_nlu.model  - Starting to train component ner_synonyms
2018-10-31 03:24:24 INFO     rasa_nlu.model  - Finished training component.
2018-10-31 03:24:24 INFO     rasa_nlu.model  - Starting to train component intent_featurizer_count_vectors
2018-10-31 03:24:24 INFO     rasa_nl

In [0]:
from rasa_nlu.model import Interpreter
import json
interpreter = Interpreter.load("./models/current/nlu")
message = "Press the first blue button"
result = interpreter.parse(message)
print(json.dumps(result, indent=2))

/usr/local/lib/python3.6/dist-packages/rasa_nlu/extractors/entity_synonyms.py:85: UserWarning: Failed to load synonyms file from './models/current/nlu/entity_synonyms.json'
  "".format(entity_synonyms_file))


INFO:tensorflow:Restoring parameters from ./models/current/nlu/intent_classifier_tensorflow_embedding.ckpt
{
  "intent": {
    "name": "button",
    "confidence": 0.8904086351394653
  },
  "entities": [
    {
      "start": 10,
      "end": 15,
      "value": "first",
      "entity": "position",
      "confidence": 0.5401163970421585,
      "extractor": "ner_crf"
    }
  ],
  "intent_ranking": [
    {
      "name": "button",
      "confidence": 0.8904086351394653
    },
    {
      "name": "mood_unhappy",
      "confidence": 0.2628970742225647
    }
  ],
  "text": "Press the first blue button"
}
